In [2]:
# Reload and changes to module without having to restart kernel.
%load_ext autoreload
%autoreload 2

### Imports

In [60]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *

import scipy.stats

import requests

### Load provided data

In [61]:
JGI_DATA = pd.read_excel('../loneliness-data/jgi_data.xlsx', sheet_name=1)

In [62]:
JGI2016, JGI2017, JGI2018 = JGI_DATA.groupby('Year')

In [63]:
# For now, just set the data from 2018 as the target variable
all_data = JGI2018[1].set_index('pcstrip')

In [64]:
JGI2018[1].columns = JGI2018[1].columns.str.replace(" ", "")

In [65]:
# test = JGI2018[1].merge(features, left_on='pcstrip', right_index=True)

In [66]:
# features_lad = test.groupby('laua').mean()[['target',
#        'A1', 'B1', 'C1', 'C2', 'D1', 'D2', 'E1', 'E2', 'F1', 'F2', 'region_1',
#        'region_2', 'region_3', 'region_4', 'region_5', 'region_6', 'region_7',
#        'region_8', 'region_9', 'index_multiple_deprivation', '0-9', '10-24',
#        '25-49', '50-79', '80+', 'nearby_schools', 'Academy', 'College',
#        'Independent School', 'Maintained School', 'Special School', 'in_0-9',
#        'in_10-24', 'in_25-49', 'in_50-79', 'in_80+', 'out_0-9', 'out_10-24',
#        'out_25-49', 'out_50-79', 'out_80+', 'moves_in_zscore',
#        'moves_out_zscore', 'schools_many', 'prop_score', 'norm_0-9',
#        'norm_10-24', 'norm_25-49', 'norm_50-79', 'norm_80+', 'ru_rank',
#        'tot_patients', 'nearby_schools_zscore', 'norm_in_0-9', 'norm_in_10-24',
#        'norm_in_25-49', 'norm_in_50-79', 'norm_in_80+', 'norm_out_0-9',
#        'norm_out_10-24', 'norm_out_25-49', 'norm_out_50-79', 'norm_out_80+']]

# Rural/Urban index, Region of countries feature

In [67]:
features = pd.DataFrame(index=all_data.index)

features['target'] = all_data['loneills']
features['target2'] = JGI2018[1]['socialanxiety_zscore'].values

In [68]:
to_num = {"A1":10, "B1":9, "C1":8, "C2":7,"D1":6, "D2":5, "E1":4,"E2":3,"F1":2,"F2":1}
features['ru_rank'] = all_data.ru11ind.apply(lambda x: to_num[x])

In [69]:

rural_urban = pd.get_dummies(all_data.ru11ind)
features = pd.concat([features, rural_urban], axis=1)

country_region = pd.get_dummies(all_data.rgn)
country_region.columns = 'region_'+country_region.columns.str[-1:]
features = pd.concat([features, country_region], axis=1)

features['index_multiple_deprivation'] = all_data.imd

In [70]:
country_region

,region_1,region_2,region_3,region_4,region_5,region_6,region_7,region_8,region_9
pcstrip,,,,,,,,,
AL100BS,0,0,0,0,0,1,0,0,0
AL100NL,0,0,0,0,0,1,0,0,0
AL108HP,0,0,0,0,0,1,0,0,0
AL13FY,0,0,0,0,0,1,0,0,0
AL13HD,0,0,0,0,0,1,0,0,0
AL13JB,0,0,0,0,0,1,0,0,0
AL14JE,0,0,0,0,0,1,0,0,0
AL21ES,0,0,0,0,0,1,0,0,0
AL21EU,0,0,0,0,0,1,0,0,0


# GP patients feature

In [71]:
p_data = pd.read_csv('../loneliness-data/gp-patients/gp-reg-pat-prac-quin-age-feb-18.csv', low_memory=False)

In [72]:
# Our data is for GP practices, remove ALL ages as we want to group by age ranges
p_data = p_data[(p_data['AGE_GROUP_5'] != 'ALL')]

# Make postcodes consistant with JGI data (no space between PC)
p_data.POSTCODE = p_data.POSTCODE.str.replace(" ", "")

In [73]:
# Find earliest age in each bracket for groupby operation (e.g. 10-14 -> 10, 25-29 -> 25)
p_data['year0'] = p_data.AGE_GROUP_5.str.split("_|\+").apply(lambda z: int(z[0]))

In [74]:
# Group each column to year bands 0-19, 20-39, ... 80+
ind = np.digitize(p_data['year0'].values,[0, 10, 25, 50, 80])

# For each year band group, group further into postcodes group and then concatenate
# results to get number of patients for each yr band in each postcode
patients = pd.concat([v.groupby('POSTCODE')['NUMBER_OF_PATIENTS'].sum() for (k, v) in p_data.groupby(ind)], axis=1)

patients.columns = ['0_9','10_24','25_49','50_79','80p']
patients.columns = 'gp'+patients.columns

patients.head()

features[patients.columns] = patients

In [75]:
features['tot_patients'] = patients.sum(axis=1)

In [76]:
patients_norm = patients.apply(lambda z: z / patients.sum(axis=1))

patients_norm.head()

features["gp_norm_"+patients_norm.columns] = patients_norm

# Number of schools in 10km radius feature

In [77]:
edu_SPINE = pd.read_csv("../loneliness-data/ofsted-data/england_spine.csv")

In [78]:
edu_SPINE = edu_SPINE[~pd.isnull(edu_SPINE['POSTCODE'])]

In [79]:
edu_SPINE['MINORGROUP'].value_counts()

edu_SPINE['MINORGROUP'] = edu_SPINE['MINORGROUP'].str.replace(" ", "")

In [80]:
schools = edu_SPINE.POSTCODE.dropna().str.split(" ").apply(lambda z: z[0]).value_counts().sort_index()

In [81]:
schools = edu_SPINE.POSTCODE.dropna().str.split(" ").apply(lambda z: z[0]).value_counts().sort_index()
result_cache = {}
def nearby_schools(pc):
    pc_outcode = pc[0:-3]
    if pc_outcode in result_cache.keys():
        return result_cache[pc_outcode]
    else:
        try:
            r = requests.get("https://api.postcodes.io/outcodes/"+pc_outcode+"/nearest", {'radius': 10000, 'limit': 100})     
            idx = [x['outcode'] for x in r.json()['result']]
            result_cache[pc_outcode] = schools[schools.index.isin(idx)].sum()
        except KeyError:
            return None
    return result_cache[pc_outcode]

In [82]:
features['nearby_schools'] = pd.Series(all_data.index, index=all_data.index).apply(lambda x: nearby_schools(x))

In [83]:
features['nearby_schools_zscore'] = (features['nearby_schools'] - features['nearby_schools'].mean()) / features['nearby_schools'].std()

# Proportion of schools in 10km radius feature

In [84]:
schools_detailed = edu_SPINE.groupby(edu_SPINE.POSTCODE.dropna().str.split(" ").apply(lambda z: z[0]))

In [85]:
schools_detailed = schools_detailed.apply(lambda z: z.MINORGROUP.value_counts(" ")).reset_index().pivot(columns = 'level_1', index='POSTCODE', values='MINORGROUP').fillna(0)

In [86]:
features = features.merge(schools_detailed, left_on=features.index.str[:-3], right_index=True)

In [87]:
del features['key_0']

# Migration features

In [88]:
d = pd.read_csv("../loneliness-data/2017_la.csv", index_col='Code')

d.head()

,Estimated Population mid-2017
Code,
K02000001,66040229
K03000001,64169395
K04000001,58744595
E92000001,55619430
E12000001,2644727


In [89]:
od1 = pd.read_csv("../loneliness-data/Detailed_Estimates_2017_Dataset_1.csv")
od2 = pd.read_csv("../loneliness-data/Detailed_Estimates_2017_Dataset_2.csv")

In [90]:
od = pd.concat([od1, od2])

In [91]:
moves_to = od.groupby('InLA').apply(lambda v: v.groupby(np.digitize(v['Age'].values,[0, 10, 25, 50, 80])).sum().T)

moves_to = moves_to.reset_index()
moves_to = moves_to[moves_to.loc[:,'level_1'] == 'Age'].set_index("InLA")
moves_to = moves_to.drop("level_1", axis=1)

moves_to.columns = ['in_0_9','in_10_24','in_25_49','in_50_79','in_80p']

# moves_to = moves_to.apply(lambda z: z / z.sum(), axis=1)

In [92]:
in_norm_by_pop = moves_to / d.loc[moves_to.index].values

in_norm_by_pop.columns = "norm_"+in_norm_by_pop.columns

In [93]:
moves_to.head()

,in_0_9,in_10_24,in_25_49,in_50_79,in_80p
InLA,,,,,
E06000001,849.0,7149.0,18681.0,15795.0,2566.0
E06000002,1333.0,18183.0,31654.0,22406.0,6680.0
E06000003,1115.0,9340.0,26881.0,33731.0,7114.0
E06000004,1631.0,17598.0,39442.0,29245.0,6980.0
E06000005,1160.0,9157.0,27781.0,29388.0,7086.0


In [94]:
moves_from = od.groupby('OutLA').apply(lambda v: v.groupby(np.digitize(v['Age'].values,[0, 10, 25, 50, 80])).sum().T)

moves_from = moves_from.reset_index()
moves_from = moves_from[moves_from.loc[:,'level_1'] == 'Age'].set_index("OutLA")
moves_from = moves_from.drop("level_1", axis=1)

moves_from.columns = ['out_0_9','out_10_24','out_25_49','out_50_79','out_80p']

# moves_from = moves_from.apply(lambda z: z / z.sum(), axis=1)

In [95]:
out_norm_by_pop = moves_from / d.loc[moves_from.index].values

out_norm_by_pop.columns = "norm_"+out_norm_by_pop.columns

In [96]:
out_norm_mig = JGI2018[1][['pcstrip', 'laua']].set_index(JGI2018[1]['pcstrip'], drop=True).merge(out_norm_by_pop, left_on='laua', right_index=True)
out_norm_mig = out_norm_mig[['norm_out_0_9', 'norm_out_10_24', 'norm_out_25_49',
       'norm_out_50_79', 'norm_out_80p']]


in_norm_mig = JGI2018[1][['pcstrip', 'laua']].set_index(JGI2018[1]['pcstrip'], drop=True).merge(in_norm_by_pop, left_on='laua', right_index=True)

in_norm_mig = in_norm_mig[['norm_in_0_9', 'norm_in_10_24', 'norm_in_25_49',
       'norm_in_50_79', 'norm_in_80p']]

In [97]:
features[in_norm_mig.columns] = in_norm_mig
features[out_norm_mig.columns] = out_norm_mig

In [98]:
moves_in = JGI2018[1][['pcstrip', 'laua']].set_index(JGI2018[1]['pcstrip'], drop=True).merge(moves_to, left_on='laua', right_index=True)
moves_out = JGI2018[1][['pcstrip', 'laua']].set_index(JGI2018[1]['pcstrip'], drop=True).merge(moves_from, left_on='laua', right_index=True)

moves_in = moves_in[['in_0_9', 'in_10_24', 'in_25_49', 'in_50_79','in_80p']]
moves_out = moves_out[['out_0_9', 'out_10_24', 'out_25_49', 'out_50_79','out_80p']]

In [99]:
features[moves_in.columns] = moves_in
features[moves_out.columns] = moves_out

In [100]:
moves_zscore = moves_in.sum(axis=1)
moves_zscore = (moves_zscore - moves_zscore.mean()) / moves_zscore.std()
features['moves_in_zscore'] = moves_zscore


moves_zscore = moves_out.sum(axis=1)
moves_zscore = (moves_zscore - moves_zscore.mean()) / moves_zscore.std()
features['moves_out_zscore'] = moves_zscore

In [101]:
features

,target,target2,ru_rank,A1,B1,C1,C2,D1,D2,E1,...,in_25_49,in_50_79,in_80p,out_0_9,out_10_24,out_25_49,out_50_79,out_80p,moves_in_zscore,moves_out_zscore
pcstrip,,,,,,,,,,,,,,,,,,,,,
AL100BS,0.616736,0.937151,8,0,0,1,0,0,0,0,...,52502.0,32048.0,10517.0,1690.0,36624.0,62514.0,44352.0,12340.0,-0.439573,-0.245584
AL100NL,1.139326,0.796187,8,0,0,1,0,0,0,0,...,52502.0,32048.0,10517.0,1690.0,36624.0,62514.0,44352.0,12340.0,-0.439573,-0.245584
AL108HP,-0.487490,0.587106,8,0,0,1,0,0,0,0,...,52502.0,32048.0,10517.0,1690.0,36624.0,62514.0,44352.0,12340.0,-0.439573,-0.245584
AL13FY,-1.439142,-1.168025,8,0,0,1,0,0,0,0,...,68783.0,33399.0,14636.0,1919.0,19761.0,71165.0,58917.0,13121.0,-0.412185,-0.165714
AL13HD,0.655981,-0.084793,8,0,0,1,0,0,0,0,...,68783.0,33399.0,14636.0,1919.0,19761.0,71165.0,58917.0,13121.0,-0.412185,-0.165714
AL13JB,1.968743,0.975865,8,0,0,1,0,0,0,0,...,68783.0,33399.0,14636.0,1919.0,19761.0,71165.0,58917.0,13121.0,-0.412185,-0.165714
AL14JE,-2.169246,-1.075872,8,0,0,1,0,0,0,0,...,68783.0,33399.0,14636.0,1919.0,19761.0,71165.0,58917.0,13121.0,-0.412185,-0.165714
AL21ES,-1.653688,-0.419309,8,0,0,1,0,0,0,0,...,68783.0,33399.0,14636.0,1919.0,19761.0,71165.0,58917.0,13121.0,-0.412185,-0.165714
AL21EU,-1.369429,-0.406214,8,0,0,1,0,0,0,0,...,68783.0,33399.0,14636.0,1919.0,19761.0,71165.0,58917.0,13121.0,-0.412185,-0.165714


In [102]:
features.dropna().to_csv("features2.csv")

# Features array at the LAD level

In [ ]:
# test = JGI2018[1].merge(features, left_on='pcstrip', right_index=True)

In [ ]:
# features_lad = test.groupby('laua').mean()[['target',
#        'A1', 'B1', 'C1', 'C2', 'D1', 'D2', 'E1', 'E2', 'F1', 'F2', 'region_1',
#        'region_2', 'region_3', 'region_4', 'region_5', 'region_6', 'region_7',
#        'region_8', 'region_9', 'index_multiple_deprivation', '0-9', '10-24',
#        '25-49', '50-79', '80+', 'nearby_schools', 'Academy', 'College',
#        'Independent School', 'Maintained School', 'Special School', 'in_0-9',
#        'in_10-24', 'in_25-49', 'in_50-79', 'in_80+', 'out_0-9', 'out_10-24',
#        'out_25-49', 'out_50-79', 'out_80+', 'moves_in_zscore',
#        'moves_out_zscore', 'schools_many', 'prop_score', 'norm_0-9',
#        'norm_10-24', 'norm_25-49', 'norm_50-79', 'norm_80+', 'ru_rank',
#        'tot_patients', 'nearby_schools_zscore', 'norm_in_0-9', 'norm_in_10-24',
#        'norm_in_25-49', 'norm_in_50-79', 'norm_in_80+', 'norm_out_0-9',
#        'norm_out_10-24', 'norm_out_25-49', 'norm_out_50-79', 'norm_out_80+']]